FONTE: https://www.gov.br/secretariageral/pt-br/acesso-a-informacao/informacoes-classificadas-e-desclassificadas

In [1]:
# lib
import pandas as pd
from google.oauth2 import service_account

In [2]:
# read csv
df = pd.read_csv('data_cartao.csv', sep=',', encoding='latin-1')

# drops and regex
df = df.drop(df.columns[[3,7]], axis=1).drop([113340, 113341]) 
df['cpf_cnpj'] = df['cpf_cnpj'].str.replace(".", "", regex=True).replace(",", "", regex=True).fillna(0)
df['valor'] = df['valor'].replace('[R&&$]', "", regex=True).replace('[.]', "", regex=True).replace(',', ".", regex=True).replace('[ &&-]', "0", regex=True)
df

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\dtypes\inference.py:177: FutureWarning: Possible set intersection at position 2
  re.compile(obj)


,data,cpf_cnpj,empresa,valor,tipo,categoria
0,02/01/2003,31349202000177,JW TRANSPORTADORA TUROSTICA,1696.90,D,LOCACAO DE MEIOS DE TRANSPORTE
1,02/01/2003,31349202000177,JW TRANSPORTADORA TUROSTICA,1336.60,D,LOCACAO DE MEIOS DE TRANSPORTE
2,02/01/2003,31349202000177,JW TRANSPORTADORA TUROSTICA,1396.43,D,LOCACAO DE MEIOS DE TRANSPORTE
3,03/01/2003,592717000170,FLORES ALVORADA,8585.00,C,OUTROS SERVIEOS DE TERCEIROS-PESSOA JURODICA
4,03/01/2003,839308000125,CRISTAL LIMPEZA - COMERCIO DE MATERIAIS PARA L...,251.50,C,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZAECO
...,...,...,...,...,...,...
113335,03/12/2022,17261661005728,OUTBACK STEAKHOUSE RESTAURANTES BRASIL S/A,179.5,C,FORNECIMENTO DE ALIMENTAECO
113336,03/12/2022,39879944000116,BIOMUNDO 105 NORTE COMERCIO DE ALIMENTOS,54.95,C,GENEROS DE ALIMENTAECO
113337,04/12/2022,47508411094975,COMPANHIA BRASILEIRA DE DISTRIBUIECO,518.07,C,GENEROS DE ALIMENTAECO
113338,04/12/2022,12550441000146,RESTAURANTE SOHO,330.52,C,FORNECIMENTO DE ALIMENTAECO


In [3]:
# count para validar se todos os registros estão compatíveis 
df.count()

data         113340
cpf_cnpj     113340
empresa      113340
valor        113340
tipo         113340
categoria    113340
dtype: int64

In [4]:
# classificação dos dados
df.data = pd.to_datetime(df.data, format = '%d/%m/%Y').dt.date
df['cpf_cnpj'] = df['cpf_cnpj'].astype('int64')
df['valor'] = df['valor'].astype(float)
df['empresa'] = df['empresa'].astype(pd.StringDtype())
df['tipo'] = df['tipo'].astype(pd.StringDtype())
df['categoria'] = df['categoria'].astype(pd.StringDtype())
df

,data,cpf_cnpj,empresa,valor,tipo,categoria
0,2003-01-02,31349202000177,JW TRANSPORTADORA TUROSTICA,1696.90,D,LOCACAO DE MEIOS DE TRANSPORTE
1,2003-01-02,31349202000177,JW TRANSPORTADORA TUROSTICA,1336.60,D,LOCACAO DE MEIOS DE TRANSPORTE
2,2003-01-02,31349202000177,JW TRANSPORTADORA TUROSTICA,1396.43,D,LOCACAO DE MEIOS DE TRANSPORTE
3,2003-01-03,592717000170,FLORES ALVORADA,8585.00,C,OUTROS SERVIEOS DE TERCEIROS-PESSOA JURODICA
4,2003-01-03,839308000125,CRISTAL LIMPEZA - COMERCIO DE MATERIAIS PARA L...,251.50,C,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZAECO
...,...,...,...,...,...,...
113335,2022-12-03,17261661005728,OUTBACK STEAKHOUSE RESTAURANTES BRASIL S/A,179.50,C,FORNECIMENTO DE ALIMENTAECO
113336,2022-12-03,39879944000116,BIOMUNDO 105 NORTE COMERCIO DE ALIMENTOS,54.95,C,GENEROS DE ALIMENTAECO
113337,2022-12-04,47508411094975,COMPANHIA BRASILEIRA DE DISTRIBUIECO,518.07,C,GENEROS DE ALIMENTAECO
113338,2022-12-04,12550441000146,RESTAURANTE SOHO,330.52,C,FORNECIMENTO DE ALIMENTAECO


In [5]:
# confirmação dos types
df.dtypes

data          object
cpf_cnpj       int64
empresa       string
valor        float64
tipo          string
categoria     string
dtype: object

In [6]:
credentials = service_account.Credentials.from_service_account_file(filename='GBQ.json', 
                                                                    scopes=["https://www.googleapis.com/auth/cloud-platform"])

In [7]:
df.to_gbq(credentials=credentials, 
          destination_table='cartao_corporativo.cartao_csv', 
          if_exists='replace', 
          table_schema=[{'name': 'data', 'type': 'DATE'}])

100%|██████████| 1/1 [00:00<?, ?it/s]
